The toy model
to get frame, trajectory, put on lstm

In [1]:
import torch
import torch.nn as nn
from data_classes import *

In [2]:
torch.__version__

'0.3.0.post4'

In [4]:
e = ETH()
V = Videodata(e)
empty_frame = V.getFrame(1)

In [15]:
traj = V.read_file('/home/yy/ADL4CV/sgan/datasets/hotel/train/biwi_eth_train.txt') # all trajectories
agent_id = 11
traj_agent = traj[np.where(traj[:, 1]==agent_id)]
traj_agent.shape

(20, 4)

In [46]:
traj_agent[0:8,2::].shape # sequence length = 8, num_fehatures = 2 xy, for one agent

(8, 2)

In [39]:
input_dim = 2 # num_input_features x and y
hidden_dim = 10 #? num features at a hidden state
num_layers = 1 # number of hidden states
lstm1 = nn.LSTM(input_dim, hidden_dim, num_layers)
#input1 = 

In [48]:
seq_len = 8
batch_size = 1
input1 = torch.fromarray(traj_agent[0:seq_len,2::]).view(seq_len, batch_size, -1)

AttributeError: module 'torch' has no attribute 'fromarray'

In [37]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

In [ ]:

model = LSTM(lstm_input_size, h1, batch_size=num_train, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

#####################
# Train model
#####################

hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

In [35]:
rnn = nn.LSTM(10, 20, 2)
input2 = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input2, (h0, c0))

RuntimeError: cannot call .data on a torch.Tensor: did you intend to use autograd.Variable?